In [1]:
import cv2
import json
from collections import defaultdict
import torch
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,  
    ShortSideScale,
    UniformTemporalSubsample
)
from torchvision.transforms import Compose, Lambda
import os.path as osp
import datetime


/home2/korengc/anaconda3/envs/koren/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

Using cache found in /home2/korengc/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [3]:
model

Net(
  (blocks): ModuleList(
    (0): MultiPathWayWithFuse(
      (multipathway_blocks): ModuleList(
        (0): ResNetBasicStem(
          (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
          (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
        (1): ResNetBasicStem(
          (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
          (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
      )
      (multipathway_fusion): FuseFastToSlow(
        (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), st

In [4]:
print(model.state_dict())

OrderedDict([('blocks.0.multipathway_blocks.0.conv.weight', tensor([[[[[-5.3844e-02, -6.3057e-02, -6.1203e-02,  ..., -1.1212e-01,
            -1.1657e-01, -1.0621e-01],
           [ 5.5371e-02,  3.0528e-02,  1.5675e-02,  ...,  5.2830e-03,
             7.8266e-03,  3.4884e-02],
           [ 3.6630e-02, -2.1915e-03, -9.0360e-03,  ..., -9.2592e-03,
            -7.8101e-03,  9.4551e-03],
           ...,
           [-2.6033e-02, -1.2046e-01, -1.1524e-01,  ..., -1.2753e-01,
            -1.3769e-01, -1.1609e-01],
           [-1.1417e-02, -9.7519e-02, -8.7518e-02,  ..., -8.9020e-02,
            -1.0302e-01, -8.3141e-02],
           [-1.2885e-02, -1.0498e-01, -1.0130e-01,  ..., -9.7639e-02,
            -9.2901e-02, -6.5305e-02]]],


         [[[-8.3175e-02, -2.4056e-03,  2.7757e-02,  ...,  4.2598e-02,
             2.5587e-02,  8.9542e-03],
           [ 3.0635e-02,  1.0118e-01,  1.2263e-01,  ...,  1.7646e-01,
             1.6289e-01,  1.5767e-01],
           [ 5.6234e-02,  1.2693e-01,  1.6375e-0

In [5]:
from pytorchvideo.models.slowfast import create_slowfast

# path = "/home2/korengc/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R50.pyth"
path = "/home2/korengc/.cache/torch/hub/checkpoints/SLOW_8x8_R50.pyth"
slowfast_model = create_slowfast(model_num_class=400)
slowfast_model.load_state_dict(torch.load(path, map_location=torch.device('cuda'))['model_state'])
slowfast_model.eval()

RuntimeError: Error(s) in loading state_dict for Net:
	Missing key(s) in state_dict: "blocks.0.multipathway_blocks.0.conv.weight", "blocks.0.multipathway_blocks.0.norm.weight", "blocks.0.multipathway_blocks.0.norm.bias", "blocks.0.multipathway_blocks.0.norm.running_mean", "blocks.0.multipathway_blocks.0.norm.running_var", "blocks.0.multipathway_blocks.1.conv.weight", "blocks.0.multipathway_blocks.1.norm.weight", "blocks.0.multipathway_blocks.1.norm.bias", "blocks.0.multipathway_blocks.1.norm.running_mean", "blocks.0.multipathway_blocks.1.norm.running_var", "blocks.0.multipathway_fusion.conv_fast_to_slow.weight", "blocks.0.multipathway_fusion.norm.weight", "blocks.0.multipathway_fusion.norm.bias", "blocks.0.multipathway_fusion.norm.running_mean", "blocks.0.multipathway_fusion.norm.running_var", "blocks.1.multipathway_blocks.0.res_blocks.0.branch1_conv.weight", "blocks.1.multipathway_blocks.0.res_blocks.0.branch1_norm.weight", "blocks.1.multipathway_blocks.0.res_blocks.0.branch1_norm.bia